### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens
from math import ceil
import itertools

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python', encoding="ISO-8859-1")

Explicit данные

In [4]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [5]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [6]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [7]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [10]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [12]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '360    Lion King, The (1994)',
 '2315    Babe: Pig in the City (1998)',
 '1838    Mulan (1998)',
 '2618    Tarzan (1999)',
 '1526    Hercules (1997)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [9]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [14]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [10]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [16]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '1892    Rain Man (1988)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [17]:
user_item.nonzero()

(array([   1,    1,    1, ..., 6040, 6040, 6040], dtype=int32),
 array([1193, 3408, 2355, ...,  562, 1096, 1097], dtype=int32))

In [18]:
user_item_exp = sp.coo_matrix((ratings.rating, (ratings.user_id, ratings.movie_id)))
user_item_exp_csr = user_item_exp.tocsr()

In [19]:
user_item_exp_csr.shape

(6041, 3953)

In [20]:
ratings.nunique()

user_id     6040
movie_id    3706
rating         5
dtype: int64

In [22]:
np.random.seed(0)

In [11]:
class MatrixFactorizationBase:
    _eps = 1e-10
    def __init__(self):
        self.u = None
        self.v = None
        
    def _get_similarity_score(self, example, others, n):
        norm_ex = np.linalg.norm(example)
        norm_other = np.linalg.norm(others, axis=-1)
        norm_ex = norm_ex if norm_ex != 0 else self._eps
        norm_other[norm_other == 0] = self._eps
        
        scores = others.dot(example) / (norm_ex * norm_other)
        best = np.argpartition(scores, -n)[-n:]
        return sorted(zip(best, scores[best]), key=lambda x: -x[1])
                
    def similar_items(self, item_id, n=10):
        return self._get_similarity_score(self.v[item_id], self.v, n)
        
    
    def similar_users(self, user_id, n=10):
        self._get_similarity_score(self.u[user_id], self.u, n)
        
    
    def recommend(self, user_id, user_items, n=10, filter_already_rated=True):
        rated = set()
        rated.update(user_items[user_id].indices)
        
        scores = self.v.dot(self.u[user_id])
        
        count = n + len(rated)
        if count < len(scores):
            ids = np.argpartition(scores, -count)[-count:]
            best = sorted(zip(ids, scores[ids]), key=lambda x: -x[1])
        else:
            best = sorted(enumerate(scores), key=lambda x: -x[1])
        return list(itertools.islice((rec for rec in best if rec[0] not in rated), n))
    

In [12]:
def masked_mse(sparse, u, v):
    approx = np.array([u[u_idx].dot(v[v_idx]) for u_idx, v_idx in zip(*sparse.nonzero())])
    return np.square(sparse.data - approx).mean()
    

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [67]:
class SVD(MatrixFactorizationBase):
    def __init__(self, factors, epochs, lr=0.1, reg=0.):
        super().__init__()
        self.factors = factors
        self.epochs = epochs
        self.lr = lr
        self.reg = reg
        
    def fit(self, x):
        u_dim, v_dim = x.shape
        nz_u_idx, nz_v_idx = x.nonzero()
        n_samples = nz_u_idx.shape[0]
        self.u = np.random.uniform(0, 1 / np.sqrt(self.factors), (u_dim, self.factors))
        self.v = np.random.uniform(0, 1 / np.sqrt(self.factors), (v_dim, self.factors))
        
        for epoch in range(self.epochs):
            for sample in np.random.permutation(n_samples):
                cur_u_idx, cur_v_idx = nz_u_idx[sample], nz_v_idx[sample]
                assert x[cur_u_idx, cur_v_idx] != 0, "Smth went wrong"
                err = self.u[cur_u_idx].dot(self.v[cur_v_idx]) - x[cur_u_idx, cur_v_idx]
                u_upd = self.lr * (err * self.v[cur_v_idx] + self.reg * self.u[cur_u_idx])
                v_upd = self.lr * (err * self.u[cur_u_idx] + self.reg * self.v[cur_v_idx])

                self.u[cur_u_idx] -= u_upd
                self.v[cur_v_idx] -= v_upd

            
            print(f"Epoch: {epoch}, mse: {masked_mse(x, self.u, self.v)}")
#             if self.log_every and (step % self.log_every == 0):
#                 print(f"Iteration: {step}, mse: {masked_mse(x, self.u, self.v)}")
                
    
        

In [71]:
model = SVD(factors=64, epochs=10, lr=0.01, reg=0.1)

In [72]:
model.fit(user_item_exp_csr)

Epoch: 0, mse: 0.9135560050615957
Epoch: 1, mse: 0.8607980470418793
Epoch: 2, mse: 0.8526941462845249
Epoch: 3, mse: 0.8502894052215714
Epoch: 4, mse: 0.84519905907256
Epoch: 5, mse: 0.8377083004357794
Epoch: 6, mse: 0.8291917090746495
Epoch: 7, mse: 0.8172634706584593
Epoch: 8, mse: 0.8091092308773898
Epoch: 9, mse: 0.802382146181684


In [73]:
model.u[1].dot(model.v[1])

4.218859621275769

In [74]:
get_similars(1, model)

['0    Toy Story (1995)',
 "2286    Bug's Life, A (1998)",
 '2327    Shakespeare in Love (1998)',
 '1287    When Harry Met Sally... (1989)',
 '2009    Jungle Book, The (1967)',
 '3045    Toy Story 2 (1999)',
 '899    Charade (1963)',
 "523    Schindler's List (1993)",
 '1628    Witness (1985)',
 '3294    American Graffiti (1973)']

In [75]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

In [76]:
get_recommendations(4, model)

['2434    Apple, The (Sib) (1998)',
 '2836    Sanjuro (1962)',
 '1950    Seven Samurai (The Magnificent Seven) (Shichin...',
 '1230    Bridge on the River Kwai, The (1957)',
 '49    Usual Suspects, The (1995)',
 '315    Shawshank Redemption, The (1994)',
 '1132    Wrong Trousers, The (1993)',
 '847    Godfather, The (1972)',
 '3269    For All Mankind (1989)',
 '735    Close Shave, A (1995)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [77]:
class ALS(MatrixFactorizationBase):
    def __init__(self, factors, iterations, reg=0.1, log_every=1):
        super().__init__()
        self.factors = factors
        self.iterations = iterations
        self.log_every = log_every
        self.reg = reg
       
    def fit(self, x):
        u_dim, v_dim = x.shape
        self.u = np.random.randn(u_dim, self.factors)
        self.v = np.random.randn(v_dim, self.factors)
        for step in range(self.iterations):
            self.u = np.linalg.solve(
                self.v.T @ self.v + self.reg * np.eye(self.factors),
                self.v.T @ x.T
            ).T
            
            self.v = np.linalg.solve(
                self.u.T @ self.u + self.reg * np.eye(self.factors),
                self.u.T @ x
            ).T
            
            if step % self.log_every == 0:
                print(f"Iteration: {step}, mse: {masked_mse(x, self.u, self.v)}")         

In [78]:
model = ALS(factors=64, iterations=100)

In [79]:
model.fit(user_item_csr)

Iteration: 0, mse: 0.6942597619192991
Iteration: 1, mse: 0.4702154288535044
Iteration: 2, mse: 0.44652894969955675
Iteration: 3, mse: 0.4366708379248764
Iteration: 4, mse: 0.4310574035748648
Iteration: 5, mse: 0.42732174996297906
Iteration: 6, mse: 0.42462892294487475
Iteration: 7, mse: 0.42259518954128855
Iteration: 8, mse: 0.42100860947005786
Iteration: 9, mse: 0.41973853281762336
Iteration: 10, mse: 0.4186992417682975
Iteration: 11, mse: 0.41783237125524053
Iteration: 12, mse: 0.41709715143917353
Iteration: 13, mse: 0.4164644956028221
Iteration: 14, mse: 0.41591324131922464
Iteration: 15, mse: 0.4154276990760125
Iteration: 16, mse: 0.4149960269770564
Iteration: 17, mse: 0.4146091365747987
Iteration: 18, mse: 0.41425994394237825
Iteration: 19, mse: 0.4139428479659723
Iteration: 20, mse: 0.4136533605884229
Iteration: 21, mse: 0.4133878405322565
Iteration: 22, mse: 0.413143298790052
Iteration: 23, mse: 0.41291725472905055
Iteration: 24, mse: 0.41270762841744724
Iteration: 25, mse: 0.41

In [82]:
model.u[1].dot(model.v[1])

0.9042674112020365

In [83]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '33    Babe (1995)',
 '2252    Pleasantville (1998)',
 '360    Lion King, The (1994)',
 '2692    Iron Giant, The (1999)',
 '591    Beauty and the Beast (1991)',
 '2225    Antz (1998)']

In [84]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '1179    Princess Bride, The (1987)',
 '2460    Planet of the Apes (1968)']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [24]:
class BPR(MatrixFactorizationBase):
    def __init__(self, factors, epochs, lr=0.1, reg=0.01):
        super().__init__()
        self.factors = factors
        self.epochs = epochs
        self.lr = lr
        self.reg = reg
       
    def fit(self, x):
        u_dim, v_dim = x.shape
        nz_u_idx, nz_v_idx = x.nonzero()
        n_samples = nz_u_idx.shape[0]
        self.u = np.random.uniform(0, 1 / np.sqrt(self.factors), (u_dim, self.factors))
        self.v = np.random.uniform(0, 1 / np.sqrt(self.factors), (v_dim, self.factors))
        for epoch in range(self.epochs):
            for step, sample in enumerate(np.random.permutation(n_samples)):
                cur_u_idx, pos_v_idx = nz_u_idx[sample], nz_v_idx[sample]
                neg_v_idx = np.random.choice(np.setdiff1d(np.arange(v_dim), nz_v_idx[nz_u_idx == cur_u_idx]))
                assert (x[cur_u_idx, neg_v_idx] == 0) and (x[cur_u_idx, pos_v_idx] != 0), "SMTH WENT WRONG"
                x_uij = self.u[cur_u_idx].dot(self.v[pos_v_idx] - self.v[neg_v_idx])
                sigm = np.exp(-x_uij) / (1. + np.exp(-x_uij))
#                 print(x_uij, common_term)
                u_upd = sigm * (self.v[pos_v_idx] - self.v[neg_v_idx]) + self.reg * self.u[cur_u_idx]
                v_pos_upd = sigm * self.u[cur_u_idx] + self.reg * self.v[pos_v_idx]
                v_neg_upd = - sigm * self.u[cur_u_idx] + self.reg * self.v[neg_v_idx]
                
                self.u[cur_u_idx] += self.lr * u_upd
                self.v[pos_v_idx] += self.lr * v_pos_upd
                self.v[neg_v_idx] += self.lr * v_neg_upd
                
#                 if step % 10000 == 0:
#                     print(step, masked_mse(x, self.u, self.v))
                    
            print(f"Epoch: {epoch}, mse: {masked_mse(x, self.u, self.v)}")
                                                          
        

In [29]:
model = BPR(factors=64, epochs=5, lr=0.001, reg=0.001)

In [30]:
model.fit(user_item_csr)

Epoch: 0, mse: 0.4705233858185092
Epoch: 1, mse: 0.3899968255859105
Epoch: 2, mse: 0.3223896696458348
Epoch: 3, mse: 0.2717363670529003
Epoch: 4, mse: 0.24381488254539677


In [31]:
get_similars(1, model)

['0    Toy Story (1995)',
 '2789    American Beauty (1999)',
 '108    Braveheart (1995)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1180    Raiders of the Lost Ark (1981)',
 '1179    Princess Bride, The (1987)',
 '1959    Saving Private Ryan (1998)',
 '847    Godfather, The (1972)',
 "523    Schindler's List (1993)"]

In [32]:
get_recommendations(4, model)

['2789    American Beauty (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '589    Silence of the Lambs, The (1991)',
 '2693    Sixth Sense, The (1999)',
 '2502    Matrix, The (1999)',
 '585    Terminator 2: Judgment Day (1991)',
 '604    Fargo (1996)',
 "523    Schindler's List (1993)",
 '315    Shawshank Redemption, The (1994)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)']

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [42]:
class WARP(MatrixFactorizationBase):
    def __init__(self, factors, epochs, lr=0.1, reg=0.01, m=1, max_negs=10):
        super().__init__()
        self.factors = factors
        self.epochs = epochs
        self.lr = lr
        self.reg = reg
        self.m = m
        self.max_negs = max_negs
        
    def fit(self, x):
        u_dim, v_dim = x.shape
        nz_u_idx, nz_v_idx = x.nonzero()
        n_samples = nz_u_idx.shape[0]
        self.u = np.random.uniform(0, 1 / np.sqrt(self.factors), (u_dim, self.factors))
        self.v = np.random.uniform(0, 1 / np.sqrt(self.factors), (v_dim, self.factors))
        for epoch in range(self.epochs):
            for step, sample in enumerate(np.random.permutation(n_samples)):
                cur_u_idx, pos_v_idx = nz_u_idx[sample], nz_v_idx[sample]
                score_pos = self.u[cur_u_idx].dot(self.v[pos_v_idx])
                for i in range(1, self.max_negs + 1):
                    neg_v_idx = np.random.choice(np.setdiff1d(np.arange(v_dim), nz_v_idx[nz_u_idx == cur_u_idx]))
                    assert (x[cur_u_idx, neg_v_idx] == 0) and (x[cur_u_idx, pos_v_idx] != 0), "SMTH WENT WRONG"
                    score_neg = self.u[cur_u_idx].dot(self.v[neg_v_idx])
                    if self.m + score_neg - score_pos > 0:
                        approx_rank = self.max_negs // i
                        loss = np.log(approx_rank)
                        u_upd = loss * (self.v[neg_v_idx] - self.v[pos_v_idx]) + self.reg * self.u[cur_u_idx]
                        v_pos_upd = - loss * self.u[cur_u_idx] + self.reg * self.v[pos_v_idx]
                        v_neg_upd = loss * self.u[cur_u_idx] + self.reg * self.v[neg_v_idx]
                        
                        self.u[cur_u_idx] -= self.lr * u_upd
                        self.v[pos_v_idx] -= self.lr * v_pos_upd
                        self.v[neg_v_idx] -= self.lr * v_neg_upd
                        
#                 if step % 10000 == 0:
#                     print(step, masked_mse(x, self.u, self.v))
            
            print(f"Epoch: {epoch}, mse: {masked_mse(x, self.u, self.v)}")

In [43]:
model = WARP(factors=64, epochs=5, lr=1e-3, reg=1e-3)
model.fit(user_item_csr)

Epoch: 0, mse: 0.5064292069512178


KeyboardInterrupt: 

In [44]:
get_similars(1, model)

['0    Toy Story (1995)',
 '108    Braveheart (1995)',
 '2789    American Beauty (1999)',
 '1180    Raiders of the Lost Ark (1981)',
 '352    Forrest Gump (1994)',
 '585    Terminator 2: Judgment Day (1991)',
 '1543    Contact (1997)',
 '604    Fargo (1996)',
 '3339    Erin Brockovich (2000)',
 '3509    Gladiator (2000)']

In [45]:
get_recommendations(4, model)

['2789    American Beauty (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '589    Silence of the Lambs, The (1991)',
 '2502    Matrix, The (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 "523    Schindler's List (1993)",
 '2693    Sixth Sense, The (1999)',
 '315    Shawshank Redemption, The (1994)',
 '604    Fargo (1996)',
 '847    Godfather, The (1972)']